In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("example_app") \
    .master("local[*]") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/02 12:22:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/06/02 16:39:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 11569642 ms exceeds timeout 120000 ms
25/06/02 16:39:13 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/02 16:39:13 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from datetime import datetime

spark = SparkSession.builder.appName("CountryRankingChange").getOrCreate()

# fb_actual_users
user_data = [
    (1, "Alice", "active", "USA"),
    (2, "Bob", "active", "India"),
    (3, "Charlie", "inactive", "USA"),
    (4, "David", "active", "UK"),
    (5, "Eve", "active", "India"),
    (6, "Frank", "active", "UK"),
    (7, "Grace", "active", "Canada")
]

user_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("status", StringType(), True),
    StructField("country", StringType(), True)
])

fb_actual_users = spark.createDataFrame(user_data, schema=user_schema)

# fb_comment_counts
comment_data = [
    (1, datetime(2021, 12, 5, 10), 5),
    (2, datetime(2021, 12, 10, 12), 8),
    (3, datetime(2021, 12, 15, 14), 2),
    (4, datetime(2021, 12, 20, 16), 3),
    (7, datetime(2021, 12, 15, 13), 4),
    (5, datetime(2022, 1, 5, 11), 7),
    (6, datetime(2022, 1, 10, 9), 7),
    (7, datetime(2022, 1, 15, 13), 6),
    (1, datetime(2022, 1, 20, 15), 3)
]

# 2021: india - 8 , usa - 7 , canada - 4, uk - 3

# 2022 : india -7, uk -7   , canada - 6 , usa -3 
# rank: 1,1,3,4
#dense_rank: 1,1,2,3



comment_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("created_dt", TimestampType(), True),
    StructField("no_comments", IntegerType(), True)
])

fb_comment_counts = spark.createDataFrame(comment_data, schema=comment_schema)


25/05/30 13:17:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# Which countries have risen in rankings based on the number 
# of comments between December 2021 vs jan 2022(Avoid gaps between ranks)

from pyspark.sql.functions import col,sum, dense_rank
from pyspark.sql.window import Window

fb_actual_users.createOrReplaceTempView('users')

fb_comment_counts.createOrReplaceTempView('counts')

spark.sql("""
          with deck_2021 as (
          select country, sum(no_comments) as sum1, dense_rank() over (order By sum(no_comments) desc) as rank from users u join counts c on u.user_id = c.user_id 
          where c.created_dt between date'2021-12-01' and date'2021-12-31'
          group by 1 
          ),
         deck_2022(
          select country, sum(no_comments) as sum1, dense_rank() over (order By sum(no_comments) desc) as rank from users u join counts c on u.user_id = c.user_id 
          where c.created_dt between date'2022-01-01' and date'2022-01-31'
          group by 1 
          )

          select d2.country,d2.sum1  from deck_2021 d1 
          join deck_2022 d2
          on d1.country = d2.country 
          and d2.rank < d1.rank
          
          """)
# .show()

window_spec = Window.orderBy(sum(col('no_comments')).desc())
deck_2021 = fb_actual_users.join(fb_comment_counts,'user_id',how='left') \
    .filter("created_dt between date'2021-12-01' and date'2021-12-31'") \
        .groupBy(col('country')) \
        .agg(dense_rank().over(window_spec).alias("rank21")
)

deck_2022 = fb_actual_users.join(fb_comment_counts,'user_id',how='left') \
    .filter("created_dt between date'2022-01-01' and date'2022-01-31'") \
        .groupBy(col('country')) \
        .agg(dense_rank().over(window_spec).alias("rank22")
)

deck_2021.show()
deck_2022.show()


deck_2022.join(deck_2021, "country", how='inner').filter("rank22 < rank21").select("country").show()


25/05/30 13:17:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 1

+-------+------+
|country|rank21|
+-------+------+
|  India|     1|
|    USA|     2|
| Canada|     3|
|     UK|     4|
+-------+------+



25/05/30 13:17:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 1

+-------+------+
|country|rank22|
+-------+------+
|  India|     1|
|     UK|     1|
| Canada|     2|
|    USA|     3|
+-------+------+



25/05/30 13:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 13:17:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/30 1

+-------+
|country|
+-------+
| Canada|
|     UK|
+-------+



25/05/30 19:14:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 893800 ms exceeds timeout 120000 ms
25/05/30 19:14:36 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/30 19:14:37 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
You are given two strings word1 and word2. Merge the strings by adding letters in alternating order, starting with word1. 
If a string is longer than the other, append the additional letters onto the end of the merged string.

Return the merged string.

 

Example 1:

Input: word1 = "abc", word2 = "pqr"
Output: "apbqcr"
Explanation: The merged string will be merged as so:
word1:  a   b   c
word2:    p   q   r
merged: a p b q c r
Example 2:

Input: word1 = "ab", word2 = "pqrs"
Output: "apbqrs"
Explanation: Notice that as word2 is longer, "rs" is appended to the end.
word1:  a   b 
word2:    p   q   r   s
merged: a p b q   r   s
Example 3:

Input: word1 = "abcd", word2 = "pq"
Output: "apbqcd"
Explanation: Notice that as word1 is longer, "cd" is appended to the end.
word1:  a   b   c   d
word2:    p   q 
merged: a p b q c   d

In [ ]:
str1 = 'abcqwertyui'
str2 = 'pqrs' 


res = ''
for i in range(1,max(len(str2),len(str1))+1):
    # print(i)
    res = res+str1[:1]+str2[:1]
    # print(res)
    str2 = str2[1:]
    str1 = str1[1:]
    # print(str1,str2)

res
# str1[:1]

'apbqcrqswertyui'

<h1>------ pyspark interview questions ---- <h2>

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Create SparkSession
spark = SparkSession.builder \
    .appName("Create Sales DataFrame") \
    .getOrCreate()

# Sample data
data = [
    ("East", "A", 100),
    ("East", "B", 150),
    ("East", "C", 120),
    ("West", "A", 200),
    ("West", "B", 180),
    ("West", "C", 210)
]

# Define schema
schema = StructType([
    StructField("region", StringType(), True),
    StructField("product", StringType(), True),
    StructField("revenue", IntegerType(), True)
])

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()


25/05/20 16:46:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+------+-------+-------+
|region|product|revenue|
+------+-------+-------+
|  East|      A|    100|
|  East|      B|    150|
|  East|      C|    120|
|  West|      A|    200|
|  West|      B|    180|
|  West|      C|    210|
+------+-------+-------+



In [ ]:
# Scenario:
# Find employees whose salary falls within the top 20% highest salaries in their respective departments.

# | emp_id | name    | department | salary |
# |--------|--------|-----------|--------|
# | 4      | David  | IT        | 4500   |
# | 5      | Eva    | Finance   | 5000   |


window_spec = Window.partitionBy(col('department')).over(col("salary"))


df = df.withColumn('rank',dense_rank().over(window_spec))


# git config --global user.name "hari"

# git config --global user.email "kishorehari228@gmail.com"

In [ ]:
# 1. reverse a string
str1 = ['h','e','l','l','o']

str1[::-1]

# 2. highest non repetating no :
num = [4,5,1,2,5,4,3,8,3,8,2,1]
freq = {}

for i in num: 
    if i in freq:
        freq[i] += 1
    else:
        freq[i] = 1
freq
    
non_repetating = []
for i in num:
    if freq[i] == 1:
        non_repetating.append(i)
if non_repetating:
    print(max(non_repetating))
else:
    print(None)

# 3. mask the string asdfghjkl to as*****kl

str1 = 'asdfghjkl'

print(str1[:2]+'*'* (len(str1) - 4)+str1[-2:])
# ,'*'* len(str1) - 4 , str[:-2]
# type(len(str1))



None
as*****kl


In [37]:
# 4. sql highest 3 
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Start Spark session
spark = SparkSession.builder.appName("TopProducts").getOrCreate()

# Sample data
data = [
    (1, 201, 10),
    (2, 202, 15),
    (3, 203, 20),
    (4, 201, 5),
    (5, 202, 10),
    (6, 204, 30)
]

# Define schema
columns = ["order_id", "product_id", "quantity"]

# Create DataFrame
df = spark.createDataFrame(data, columns)
df.createOrReplaceTempView('orders')

spark.sql('''
select product_id, sum(quantity) from orders group by product_id order by 2 desc limit 3
''').show()


+----------+-------------+
|product_id|sum(quantity)|
+----------+-------------+
|       204|           30|
|       202|           25|
|       203|           20|
+----------+-------------+



In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, lead
from pyspark.sql.window import Window

# Start Spark session
spark = SparkSession.builder.appName("ConsecutiveNums").getOrCreate()

# Sample data
data = [
    (1, '1'),
    (2, '1'),
    (3, '1'),
    (4, '2'),
    (5, '1'),
    (6, '1'),
    (7, '1')
]

# Define schema
columns = ["id", "num"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView('numbers')


spark.sql("""
          select num from (
select distinct num ,lag(num,1) over(order by id) as prev,lead(num,1) over( order by id ) as next from numbers 
          ) where num = prev = next
""").show()



25/05/28 08:45:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/28 08:45:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+
|num|
+---+
|  1|
+---+



25/05/28 08:45:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/28 08:45:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [67]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, lead
from pyspark.sql.window import Window

# Start Spark session
spark = SparkSession.builder.appName("employee").getOrCreate()

# Sample data
data = [
    (1, '100'),
    (2, '200'),
    (3, '300')

]

# Define schema
columns = ["id", "salary"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView('employee')

spark.sql("""
          select coalesce(max(salary), Null) from (select salary from 
(select id, salary, rank() over (order by salary desc) as rank from employee )where rank = 2 )
          """).show()


25/05/28 09:32:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/28 09:32:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------------------+
|coalesce(max(salary), NULL)|
+---------------------------+
|                        200|
+---------------------------+



25/05/28 09:32:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/28 09:32:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [71]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, lead
from pyspark.sql.window import Window

# Start Spark session
spark = SparkSession.builder.appName("employee").getOrCreate()

# Sample data
data = [
    (1, '100'),
    (2, '200'),
    (3, '300'),
    (4, '300')

]

# Define schema
columns = ["id", "score"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView('scores')

spark.sql("""
          select score, dense_rank() over (order by score desc) from scores
          """).show()

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# Question: Employee Salaries and Department Analysis
# You are given three DataFrames:
# employees
#  | emp_id | emp_name | dept_id | hire_date |
#  |--------|----------|---------|-----------|
#  | 1 | Alice | 10 | 2020-01-15|
#  | 2 | Bob | 20 | 2019-03-10|
#  | 3 | Charlie | 10 | 2021-07-23|
#  | 4 | David | 30 | 2018-06-01|
#  | 5 | Eva | 10 | 2020-11-12|
# departments
#  | dept_id | dept_name |
#  |---------|-------------|
#  | 10 | HR |
#  | 20 | IT |
#  | 30 | Finance |
# salaries
#  | emp_id | salary | effective_date |
#  |--------|--------|----------------|
#  | 1 | 70000 | 2021-01-01 |
#  | 1 | 75000 | 2022-01-01 |
#  | 2 | 90000 | 2020-01-01 |
#  | 3 | 60000 | 2022-01-01 |
#  | 5 | 65000 | 2021-01-01 |

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from pyspark.sql.types import DateType,IntegerType,StringType,StructType,StructField
from datetime import datetime


def to_date(str1):
    return datetime.strptime(str1, "%Y-%m-%d").date()


spark = SparkSession.builder.appName("hari").getOrCreate()

employees_data = [
    (1, "Alice", 10, to_date("2020-01-15")),
    (2, "Bob", 20, to_date("2019-03-10")),
    (3, "Charlie", 10, to_date("2021-07-23")),
    (4, "David", 30, to_date("2018-06-01")),
    (5, "Eva", 10, to_date("2020-11-12"))
]

employees_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("emp_name", StringType(), True),
    StructField("dept_id", IntegerType(), True),
    StructField("hire_date", DateType(), True)
])

employees_df = spark.createDataFrame(employees_data, schema=employees_schema)

# -------------------------------
# 2. DEPARTMENTS
# -------------------------------
departments_data = [
    (10, "HR"),
    (20, "IT"),
    (30, "Finance")
]

departments_schema = StructType([
    StructField("dept_id", IntegerType(), True),
    StructField("dept_name", StringType(), True)
])

departments_df = spark.createDataFrame(departments_data, schema=departments_schema)

# -------------------------------
# 3. SALARIES
# -------------------------------
salaries_data = [
    (1, 70000, to_date("2021-01-01")),
    (1, 75000, to_date("2022-01-01")),
    (2, 90000, to_date("2020-01-01")),
    (3, 60000, to_date("2022-01-01")),
    (5, 65000, to_date("2021-01-01"))
]

salaries_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("salary", IntegerType(), True),
    StructField("effective_date", DateType(), True)
])

salaries_df = spark.createDataFrame(salaries_data, schema=salaries_schema)


# Write PySpark code to:
# Get each employee’s latest salary based on effective_date.
# Join with department info.
# For each department, compute:
# Number of employees
# Average salary
# Name of the highest paid employee in that department

from pyspark.sql.window import Window

window_spec = Window.partitionBy("emp_id").orderBy(col("effective_date").desc())
window_salary = Window.partitionBy("dept_id").orderBy(col("salary").desc())


salaries_df = salaries_df.withColumn("rank", dense_rank().over(window_spec)).filter(col("rank")== lit(1))

employees_df.join(departments_df,on="dept_id", how="left").join(salaries_df, on="emp_id",how="left")\
.groupBy("dept_name").agg(count(col('emp_id').alias("count_n")),avg(col('salary').alias("avg_sal"))).show()

employees_df.join(departments_df,on="dept_id", how="left").join(salaries_df, on="emp_id",how="left")\
.withColumn("sal_rank", dense_rank().over(window_salary)).filter("sal_rank = 1").fillna({'salary':1000}) \
.withColumn("bucket", when(col("salary")>= 100000, "high")
            .when((col("salary") >= 50000) & (col("salary") < 100000), "medium").otherwise("low")).show()



+---------+------------------------+----------------------+
|dept_name|count(emp_id AS count_n)|avg(salary AS avg_sal)|
+---------+------------------------+----------------------+
|       HR|                       3|     66666.66666666667|
|  Finance|                       1|                  NULL|
|       IT|                       1|               90000.0|
+---------+------------------------+----------------------+



+------+-------+--------+----------+---------+------+--------------+----+--------+------+
|emp_id|dept_id|emp_name| hire_date|dept_name|salary|effective_date|rank|sal_rank|bucket|
+------+-------+--------+----------+---------+------+--------------+----+--------+------+
|     1|     10|   Alice|2020-01-15|       HR| 75000|    2022-01-01|   1|       1|medium|
|     2|     20|     Bob|2019-03-10|       IT| 90000|    2020-01-01|   1|       1|medium|
|     4|     30|   David|2018-06-01|  Finance|  1000|          NULL|NULL|       1|   low|
+------+-------+--------+----------+---------+------+--------------+----+--------+------+



25/06/07 08:45:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 888637 ms exceeds timeout 120000 ms
25/06/07 08:45:28 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/07 08:45:35 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [13]:
# Spark Interview Question – Real-World Problem

# 🚀 Problem Statement:
#  You're given two DataFrames:
# employees – contains employee details:
#  (employee_id, name, salary, department_id)
# departments – contains department metadata:
#  (department_id, department_name)
# 🎯 Objective:
#  Fill in the missing (null) salaries in the employees DataFrame with the average salary of that respective department.

# employees 
# +-----------+-------+------+-------------+
# |employee_id| name|salary|department_id|
# +-----------+-------+------+-------------+
# | 1| Alice| 70000| 10|
# | 2| Bob| NULL| 20|
# | 3|Charlie| 80000| 10|
# | 4| David| NULL| 30|
# | 5| Eve| 75000| 20|
# | 6| Frank| 90000| 10|
# | 7| Grace| 52000| 30|
# | 8| Hannah| 62000| 20|
# | 9| Isaac| NULL| 30|
# | 10| Jack| 71000| 20|
# +-----------+-------+------+-------------+

# departments :
# +-------------+---------------+
# |department_id|department_name|
# +-------------+---------------+
# | 10| Engineering|
# | 20| HR|
# | 30| Marketing|
# +-------------+---------------+

# ✅ Solution in PySpark:

from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("FillNullSalaries").getOrCreate()

# Data
employee_data = [
 (1, "Alice", 70000, 10),
 (2, "Bob", None, 20),
 (3, "Charlie", 80000, 10),
 (4, "David", None, 30),
 (5, "Eve", 75000, 20),
 (6, "Frank", 90000, 10),
 (7, "Grace", 52000, 30),
 (8, "Hannah", 62000, 20),
 (9, "Isaac", None, 30),
 (10, "Jack", 71000, 20)
]

department_data = [
 (10, "Engineering"),
 (20, "HR"),
 (30, "Marketing")
]


employees_df = spark.createDataFrame(employee_data, schema=["employee_id", "name", "salary", "department_id"])
departments_df = spark.createDataFrame(department_data, ["department_id", "department_name"])


dept_avg_df = employees_df.groupBy("department_id").agg(F.round(F.avg('salary'), 1).alias("avg_sal"))


employees_df.join(dept_avg_df , dept_avg_df.department_id == employees_df.department_id, how = 'left')\
.withColumn("salary", F.coalesce(F.col('salary'),F.col("avg_sal")))\
    .drop(dept_avg_df.department_id).show()







+-----------+-------+-------+-------------+-------+
|employee_id|   name| salary|department_id|avg_sal|
+-----------+-------+-------+-------------+-------+
|          1|  Alice|70000.0|           10|80000.0|
|          2|    Bob|69333.3|           20|69333.3|
|          3|Charlie|80000.0|           10|80000.0|
|          4|  David|52000.0|           30|52000.0|
|          5|    Eve|75000.0|           20|69333.3|
|          6|  Frank|90000.0|           10|80000.0|
|          7|  Grace|52000.0|           30|52000.0|
|          8| Hannah|62000.0|           20|69333.3|
|          9|  Isaac|52000.0|           30|52000.0|
|         10|   Jack|71000.0|           20|69333.3|
+-----------+-------+-------+-------------+-------+



In [17]:
# Filter out customers whose names start with the letter 'A' and display all their details using PySpark.

# Sample Data :
data = [
 (1, "Aman Gupta", 28, "Delhi"),
 (2, "Ravi Kumar", 34, "Bangalore"),
 (3, "Anjali Rai", 25, "Mumbai"),
 (4, "Priya Mehta", 30, "Hyderabad"),
 (5, "Arjun Verma", 32, "Pune"),
 (6, "Sneha Jain", 27, "Kolkata"),
 (7, "Ajay Singh", 29, "Lucknow")
]

# Column names
columns = ["CustomerID", "Name", "Age", "City"]


customer_df = spark.createDataFrame(data, columns)

customer_df.filter(F.startswith("Name",F.lit("A"))).show()

+----------+-----------+---+-------+
|CustomerID|       Name|Age|   City|
+----------+-----------+---+-------+
|         1| Aman Gupta| 28|  Delhi|
|         3| Anjali Rai| 25| Mumbai|
|         5|Arjun Verma| 32|   Pune|
|         7| Ajay Singh| 29|Lucknow|
+----------+-----------+---+-------+



25/06/09 09:26:29 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 899524 ms exceeds timeout 120000 ms
25/06/09 09:26:29 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/09 09:26:31 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
str1 = "asd hdif in a abcd in a this asd"
del1 = ['in','a','this']
# word count :

#python
count = {}
for i in [i for i in str1.split() if i not in del1]:
    c = 0
    for j in str1.split():
        if i==j:
            c+=1
    count.update({i:c})

count

# consecutive sensor reading greater than 10



# sql :

# count of characters given in a string 


{'asd': 2, 'hdif': 1, 'abcd': 1}